In [7]:
import numpy as np

Generate the possible placements. For each of these, the results can be shuffled, but we'll handle that later. 

In [8]:
placements = []

for i in range(2, 21, 2):
    placements.append((i, 12 - i//2, 18 - i//2))
    print(placements[-1])

(2, 11, 17)
(4, 10, 16)
(6, 9, 15)
(8, 8, 14)
(10, 7, 13)
(12, 6, 12)
(14, 5, 11)
(16, 4, 10)
(18, 3, 9)
(20, 2, 8)


Above, we can see that for all of the placements, there are never more than 20 rubies in a single box, so we can limit our guesses to ones that never guess over 20, since all of those would always bust.

In [9]:
# Find all possible guesses
# All configurations have at least 2 rubies
# Don't guess more than 20 for any boxes since the constraints don't allow boxes to have more than 20

guesses = []

max_in_box = 20
for i in range(2, max_in_box+1):
    for j in range(i, max_in_box+1):
        for k in range(j, max_in_box+1):
            guesses.append((i, j, k))

In [10]:
print('{} possible placements'.format(len(placements)))
print('{} possible guesses'.format(len(guesses)))

10 possible placements
1330 possible guesses


For each combination of a placement and a guess, there will be a certain outcome. We calculate the different permutations here because it's always going to be better for the placer to randomize because if it were deterministic, the guesser could pretty easily catch on and exploit that.

In [11]:
def compute_outcome(guess, placement):
    permutations = [
        (0, 1, 2),
        (0, 2, 1),
        (1, 0, 2),
        (1, 2, 0),
        (2, 0, 1),
        (2, 1, 0)
    ]

    score = 0
    for permutation in permutations:
        for i in range(3):
            if guess[i] <= placement[permutation[i]]:
                score += guess[i]

    return score / 6

We would need to compute the outcome many times, so it's smarter just to pre-compute the outcomes and store them in a numpy array.

In [12]:
outcomes = np.zeros((len(guesses), len(placements)))

for guess_num, guess in enumerate(guesses):
    for placement_num, placement in enumerate(placements):
        outcomes[guess_num, placement_num] = compute_outcome(guess, placement)

Next, we can start making the methods for computing the policy and the update. Note the difference in the fast method

In [13]:
def compute_softmax(weights):
    exp = np.exp(weights)
    exp_sum = np.sum(exp)
    out = exp / exp_sum
    return out


def update(weights, rewards, out, lr):
    grad = out * (rewards - np.dot(rewards, out))
    new_weights = weights + grad * lr
    return new_weights

def update_fast(weights, rewards, out, lr):
    # The difference is that it isn't multiplied by the probability
    # of each action occuring. Although the other version is correct,
    # it can make convergence very slow.
    #
    # We can do this because each partial derivative keeps it's sign,
    # so each parameter updates in the correct direction.
    # This is similar to how importance sampling works
    # You can also think of it as just increasing the learning rate for certain actions.
    grad = (rewards - np.dot(rewards, out))
    new_weights = weights + grad * lr
    return new_weights

Initialize the policy weights

In [18]:
# All zeros is a random policy
guess_weights = np.zeros(len(guesses), np.float32)
placement_weights = np.zeros(len(placements), np.float32)

# Set the weight for (12, 6, 12) higher to see what happens
# placement_weights[5] = 4

guess_policy = compute_softmax(guess_weights)
placement_policy = compute_softmax(placement_weights)

If you want to hold one of the policies constant, use this. You can set the policies in the previous cell

In [19]:
update_guesses = True
update_placements = True

update_func = update
# update_func = update_fast

guess_learning_rate = 0.01
placement_learning_rate = 0.001

eights = guesses.index((8, 8, 8))

Run the cell below to actually train the policy gradient. Feel free to modify the previous two cells to see the different things when changing the initial policies. Additionally, feel free to change the logging to see what actions the policies value most.

#### Here are some interesting things to try:

Hold the placement policy constant. When it\'s completely random, the optimal policy is (9, 9, 9), not (8, 8, 8).

Try changing the initial placement policy - if one placement dominates, then the guess policy changes to reflect this.

Changing the learning rates and starting policies change the convergence properties. 

#### Notes on convergence

If the guess policy converges to one policy, but then the placement policy changes to compensate, we can get in a situation where convergence drastically slows down. The probability of the previously optimal action is so high that the probability for the other actions end up being extremely close to 0. The update sizes are proportional to the probability, so they update very slowly. The action with the high probability continues to become more likely because it\'s updates are larger. We end up in a one step forward, two steps back situation where each gradient update gets us slightly closer to the new optimum, but the gradient gets smaller and it takes even more steps to get there.

To use a specific example, imagine starting with (12, 6, 12) having a 99% chance of being placed. The optimal guessing policy would be (12, 12, 12) for a score of 24. It converges to about 99.99%. The placement policy sees this and updates so that the probability of (12, 6, 12) is very low (say 1%) while the others have a probability of 11% each. On every other placement (12, 12, 12) scores a 12, but the guess policy hasn't quite converged to (12, 12, 12), so the expected value is now 11.999ish, which means that increasing the probability of (12, 12, 12) still improves the expected value. At this point a better strategy is (8, 8, 8) with an average score of 16, but imagine the probability of (8, 8, 8) is 10^-7. Comparing the two update rules p_a (r_a - E\[r\]), we get that the partial derivative for (12, 12, 12) is about 0.9999 (12 - 11.999) = 0.001ish while the partial derivative for (8, 8, 8) is 10^-7 (16 - 11.999) = 4\*10^-7. This means that the relative probability of (12, 12, 12) is increasing, not decreasing! Because the softmax is defined in terms of relative probabilities, the overall probability of (12, 12, 12) is increasing too! Eventually, if the relative probability of (8, 8, 8) goes up and the 11.999 turns into a 12.001, then (8, 8, 8) would stand a chance, but there are a lot (over 1000) bad policies that (8, 8, 8) has to beat out before this can happen. Since the updates are so small, any weight updates that occured in the beginning of training would dominate and it may take a long time before the relatively small updates that are now taking place can move the weight.

One step forward, two steps back...

Eventually, the other probabilities should even out such that (8, 8, 8) makes it\'s way to the top, but this happens very very slowly because the changes to the weight in the beginning are what dominate the total weight, and any subsequent changes are small compared to the actual weight. Still though, the relative changes in probability to whichever action is dominating, lead the gradient of the other actions to roughly follow a gemetric series, which converges to a finite value. In that approximation, which is fairly accurate, the policy literally never converges. Due to the other small changes that are occuring, the policy would still *eventually* converge. Practically, never, but still *eventually*.

This doesn't just happen with the guess policy though, it can pretty easily happen with the placement policy too. Without any modifications, policy gradient can be difficult to deal with. If we change the update method to update_fast, we can avoid some of these issues. It is discussed breifly in the method definition above and is similar to importance sampling, but that's too much to get into for now.


In [21]:

for i in range(100000000):
    if i + 1 in [0, 1, 10, 25, 100, 1000, 10000] or (i + 1) % 100000 == 0:
        print("Iteration", i + 1)
        args = np.argsort(guess_policy)[::-1]
        best_guess, next_best_guess = args[:2]
        eight_rank = np.where(args == eights)[0][0]+1
        print("Best guess: {} P={}".format(guesses[best_guess], guess_policy[best_guess]))
        print("Next guess: {} P={}".format(guesses[next_best_guess], guess_policy[next_best_guess]))
        print("Eights: {} P={}, rank={}".format(guesses[eights], guess_policy[eights], eight_rank))

        for placement in range(len(placements)):
            print(placements[placement], placement_policy[placement], np.dot(outcomes[:, placement], guess_policy))
        print("Expected game value: {}".format(np.dot(np.matmul(outcomes, placement_policy), guess_policy)))
        print()

    # train guesser
    if update_guesses:
        guess_rewards = np.matmul(outcomes, placement_policy)
    if update_placements:
        placement_rewards = np.matmul(outcomes.T, guess_policy)
    
    if update_guesses:
        guess_policy = compute_softmax(guess_weights)
        # Positive rewards for guesser because he's the one getting the extra rubies
        guess_weights = update_func(guess_weights, guess_rewards, guess_policy, guess_learning_rate)
    
    if update_placements:
        placement_policy = compute_softmax(placement_weights)
        # Negative rewards for placer because he's the one losing the extra rubies
        placement_weights = update_func(placement_weights, -placement_rewards, placement_policy, placement_learning_rate)

Iteration 1
Best guess: (8, 8, 8) P=0.9999938771287566
Next guess: (7, 8, 8) P=1.812381876591547e-07
Eights: (8, 8, 8) P=0.9999938771287566, rank=1
(2, 11, 17) 0.0004097217047644064 15.99998299568607
(4, 10, 16) 0.000826377730055524 15.999978169223493
(6, 9, 15) 0.0028423530996168705 15.999974710780203
(8, 8, 14) 1.3619773643192389e-05 23.999929349289882
(10, 7, 13) 0.9880324849194454 15.999978788253456
(12, 6, 12) 0.004585699944912663 15.999978312531622
(14, 5, 11) 0.0016406991035910218 15.999978021090634
(16, 4, 10) 0.000826377730055524 15.999978169223493
(18, 3, 9) 0.0005008689215496011 15.999977597946701
(20, 2, 8) 0.0003217970723660258 15.999978812401418
Expected game value: 16.00008773085179

Iteration 10
Best guess: (8, 8, 8) P=0.9999938771404456
Next guess: (7, 8, 8) P=1.8123784626022268e-07
Eights: (8, 8, 8) P=0.9999938771404456, rank=1
(2, 11, 17) 0.000409721312747385 15.99998299571859
(4, 10, 16) 0.0008263769397410235 15.99997816926523
(6, 9, 15) 0.0028423503872060814 15.999

KeyboardInterrupt: 

Now, below is what happens when we sample from the policies and compute an estimate of the gradient rather than doing all of that explicitly.

The first thing to notice is that it runs slower. Additionally, it converges slower. It does manage to find some good policies. (8, 8, 8) is towards the top (at least before it converges to something else) and the other actions at the top don't seem too bad either. This makes sense. Anything towards the top gets selected more often, so if something bad was towards the top, it would get moved down pretty fast, at least when the probabilites are all roughly equal. 

Still, we see the same problems as in the deterministic case. The placement policy starts to converge to an optimum. The guess policy notices this and quickly exploits it. This lead to the placement policy changing again, but by then the guess policy was so decided that it has a hard time changing.

In [22]:
def update_stochastic(actions, rewards, weights, out, lr):
    grad = np.zeros_like(weights)
    for action, reward in zip(actions, rewards):
        grad -= out * reward
        grad[action] += reward
    new_weights = weights + grad * lr / len(actions)
    return new_weights

In [31]:
guess_weights = np.zeros(len(guesses), np.float32)  # random policies
placement_weights = np.zeros(len(placements), np.float32)

guess_policy = compute_softmax(guess_weights)
placement_policy = compute_softmax(placement_weights)

In [32]:
update_guesses = True
update_placements = True

guess_learning_rate = 0.01
placement_learning_rate = 0.01

num_samples = 100

eights = guesses.index((8, 8, 8))

In [33]:
for i in range(10000000):
    if i + 1 in [0, 1, 10, 25] or (i + 1) % 500 == 0:
        print("Iteration", i + 1)
        args = np.argsort(guess_policy)[::-1]
        best_guess, next_best_guess = args[:2]
        eight_rank = np.where(args == eights)[0][0]+1
        print("Best guess: {} P={}".format(guesses[best_guess], guess_policy[best_guess]))
        print("Next guess: {} P={}".format(guesses[next_best_guess], guess_policy[next_best_guess]))
        print("Eights: {} P={}, rank={}".format(guesses[eights], guess_policy[eights], eight_rank))

        for placement in range(len(placements)):
            print(placements[placement], placement_policy[placement], np.dot(outcomes[:, placement], guess_policy))
        print("Expected game value: {}".format(np.dot(np.matmul(outcomes, placement_policy), guess_policy)))
        print()

    # train guesser
    if update_guesses or update_placements:
        guess_actions = np.zeros(num_samples, np.int32)
        placement_actions = np.zeros(num_samples, np.int32)
        rewards = np.zeros(num_samples, np.float32)
        for i in range(num_samples):
            guess_actions[i] = np.random.choice(len(guesses), p=guess_policy)
            placement_actions[i] = np.random.choice(len(placements), p=placement_policy)
            rewards[i] = outcomes[guess_actions[i], placement_actions[i]]

    if update_guesses:
        guess_weights = update_stochastic(guess_actions, rewards, guess_weights, guess_policy, guess_learning_rate)
        guess_policy = compute_softmax(guess_weights)
        # Positive rewards for guesser because he's the one getting the extra rubies

    if update_placements:
        # Negative rewards for placer because he's the one losing the extra rubies
        placement_weights = update_stochastic(placement_actions, -rewards, placement_weights, placement_policy, placement_learning_rate)
        placement_policy = compute_softmax(placement_weights)

Iteration 1
Best guess: (20, 20, 20) P=0.0007518797065131366
Next guess: (4, 9, 9) P=0.0007518797065131366
Eights: (8, 8, 8) P=0.0007518797065131366, rank=1314
(2, 11, 17) 0.1 11.526315900846386
(4, 10, 16) 0.1 10.42105273227207
(6, 9, 15) 0.1 9.631579040433278
(8, 8, 14) 0.1 9.157894825330004
(10, 7, 13) 0.1 9.000000086962235
(12, 6, 12) 0.1 9.157894825329997
(14, 5, 11) 0.1 9.63157904043328
(16, 4, 10) 0.1 10.42105273227207
(18, 3, 9) 0.1 11.526315900846388
(20, 2, 8) 0.1 12.947368546156213
Expected game value: 10.342105517197576

Iteration 10
Best guess: (6, 8, 13) P=0.0007559758960269392
Next guess: (10, 11, 15) P=0.0007550946902483702
Eights: (8, 8, 8) P=0.0007525569526478648, rank=194
(2, 11, 17) 0.09964385 11.526833094966904
(4, 10, 16) 0.098146684 10.421667429366307
(6, 9, 15) 0.10178388 9.632320629180565
(8, 8, 14) 0.101918004 9.158667287692275
(10, 7, 13) 0.0984269 9.00088300786836
(12, 6, 12) 0.101816036 9.158809466423314
(14, 5, 11) 0.10148586 9.632369836831158
(16, 4, 10) 

Iteration 6000
Best guess: (10, 10, 10) P=0.0013484470546245575
Next guess: (7, 7, 10) P=0.0013477907050400972
Eights: (8, 8, 8) P=0.0010745690669864416, rank=48
(2, 11, 17) 0.0016209584 11.905266566202041
(4, 10, 16) 0.0026772637 10.898890374247761
(6, 9, 15) 0.0061661527 10.193965696768519
(8, 8, 14) 0.029240448 9.87255147726197
(10, 7, 13) 0.9192869 9.886841215309685
(12, 6, 12) 0.029034842 10.023376382790344
(14, 5, 11) 0.006841162 10.361907937583354
(16, 4, 10) 0.0028200194 10.898890374247761
(18, 3, 9) 0.0014402434 11.652322302417215
(20, 2, 8) 0.00087208586 12.714763098881424
Expected game value: 9.909376469647153

Iteration 6500
Best guess: (7, 7, 10) P=0.0014514627400785685
Next guess: (10, 10, 10) P=0.001425502123311162
Eights: (8, 8, 8) P=0.0011027060681954026, rank=54
(2, 11, 17) 0.0013995506 11.939851229311902
(4, 10, 16) 0.0023483394 10.943909973274767
(6, 9, 15) 0.0051919883 10.24450850090944
(8, 8, 14) 0.025216797 9.938898385191964
(10, 7, 13) 0.93014455 9.9751919125750

Iteration 12500
Best guess: (7, 7, 10) P=0.006676915567368269
Next guess: (10, 10, 10) P=0.005925809033215046
Eights: (8, 8, 8) P=0.0015446697361767292, rank=74
(2, 11, 17) 0.0007038944 12.488442682137254
(4, 10, 16) 0.0012120118 11.646233580851305
(6, 9, 15) 0.0028535724 10.947568327528035
(8, 8, 14) 0.015266556 10.894962518165503
(10, 7, 13) 0.96292734 11.322363701396775
(12, 6, 12) 0.011846104 11.262710136691263
(14, 5, 11) 0.0028910942 11.401045060794178
(16, 4, 10) 0.0012942321 11.646233580851305
(18, 3, 9) 0.00061472144 11.8440092575911
(20, 2, 8) 0.0003906705 12.448061594303008
Expected game value: 11.31668524612666

Iteration 13000
Best guess: (7, 7, 10) P=0.00915725901722908
Next guess: (10, 10, 10) P=0.007390855345875025
Eights: (8, 8, 8) P=0.0016085689421743155, rank=66
(2, 11, 17) 0.00070642855 12.5678479517325
(4, 10, 16) 0.0011974417 11.744915380821716
(6, 9, 15) 0.0029479344 11.022218566397589
(8, 8, 14) 0.01651801 11.009098191735873
(10, 7, 13) 0.96148354 11.50181949584

Iteration 19500
Best guess: (7, 7, 10) P=0.9920018315315247
Next guess: (10, 10, 10) P=0.0002115416864398867
Eights: (8, 8, 8) P=1.3299076272232924e-05, rank=70
(2, 11, 17) 0.00016885453 15.975136824230274
(4, 10, 16) 0.0002693346 15.969047838534987
(6, 9, 15) 0.9937811 12.655070242632823
(8, 8, 14) 0.0012286821 17.284874939097627
(10, 7, 13) 0.00053176645 20.598054532016803
(12, 6, 12) 0.0026593304 15.967271198322125
(14, 5, 11) 0.00075794425 15.967741712137771
(16, 4, 10) 0.00030014673 15.969047838534987
(18, 3, 9) 0.00019126954 12.660605519876754
(20, 2, 8) 0.00011158174 12.663930982727678
Expected game value: 12.67875157561263

Iteration 20000
Best guess: (7, 7, 10) P=0.9923214316368103
Next guess: (10, 10, 10) P=0.00020324575598351657
Eights: (8, 8, 8) P=1.2756472642649896e-05, rank=70
(2, 11, 17) 0.00014577707 15.976131378454582
(4, 10, 16) 0.00023090512 15.970286576947357
(6, 9, 15) 0.9948185 12.655536422296189
(8, 8, 14) 0.0010319173 17.28681272392803
(10, 7, 13) 0.00044597386 

Iteration 26000
Best guess: (7, 7, 10) P=0.9953300952911377
Next guess: (10, 10, 10) P=0.00011972809443250299
Eights: (8, 8, 8) P=7.74087220634101e-06, rank=71
(2, 11, 17) 7.5311524e-05 15.985464658276852
(4, 10, 16) 0.00011671794 15.981910843624817
(6, 9, 15) 0.997695 12.659923337650113
(8, 8, 14) 0.00044578116 17.305063002261647
(10, 7, 13) 0.00020579317 20.626596641448153
(12, 6, 12) 0.00088742434 15.980875400215792
(14, 5, 11) 0.00030680775 15.981149202156566
(16, 4, 10) 0.00013137067 15.981910843624817
(18, 3, 9) 8.5636646e-05 12.663145604081317
(20, 2, 8) 4.9996266e-05 12.665073074662736
Expected game value: 12.668673076211213

Iteration 26500
Best guess: (7, 7, 10) P=0.9953650236129761
Next guess: (10, 10, 10) P=0.00011839531362056732
Eights: (8, 8, 8) P=7.678981091885362e-06, rank=72
(2, 11, 17) 7.374273e-05 15.985573401651934
(4, 10, 16) 0.000114397764 15.98204628745778
(6, 9, 15) 0.9977597 12.659976758241061
(8, 8, 14) 0.00043045956 17.305276244849313
(10, 7, 13) 0.0002011946

KeyboardInterrupt: 